<a href="https://colab.research.google.com/github/hamednasr/transformers/blob/main/Decoder_transformers_in_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 29.7 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset
from datetime import datetime

In [3]:
class CausalSelfAttention(nn.Module):
  def __init__(self, d_k, d_v, d_model, n_heads, max_len):
    super().__init__()

    self.d_k = d_k
    self.d_v = d_v
    self.n_heads = n_heads

    self.W_q = nn.Linear(d_model,d_k*n_heads)
    self.W_k = nn.Linear(d_model,d_k*n_heads)
    self.W_v = nn.Linear(d_model,d_v*n_heads)

    self.fc = nn.Linear(d_v*n_heads, d_model)

    cm = torch.tril(torch.ones(max_len,max_len))
    self.register_buffer('causal_mask', cm.view(1,1,max_len,max_len))

  def forward(self, X, pad_mask=None): # X could be q,k,v that are different
    Q = self.W_q(X)  # N × T × h*d_k
    K = self.W_k(X)  # N × T × h*d_k
    V = self.W_v(X)  # N × T × h*d_v

    N = Q.shape[0]
    T = Q.shape[1]

    Q = Q.view(N, T, self.n_heads, self.d_k).transpose(1,2) # N × T × h*d_k -->> N × h × T × d_k
    K = K.view(N, T, self.n_heads, self.d_k).transpose(1,2) # N × T × h*d_k -->> N × h × T × d_k
    V = V.view(N, T, self.n_heads, self.d_k).transpose(1,2) # N × T × h*d_k -->> N × h × T × d_k

    AttentionScores = Q @ K.transpose(2,3) / np.sqrt(self.d_k) #  N × h × T × T

    if pad_mask is not None:
      pad_mask= torch.unsqueeze(pad_mask, 1)
      pad_mask= torch.unsqueeze(pad_mask, 1)
      AttentionScores = AttentionScores.masked_fill(pad_mask == 0, float('-inf'))

    AttentionScores = AttentionScores.masked_fill(self.causal_mask[:,:,:T,:T] == 0, float('-inf'))

    AttentionWeights = F.softmax(AttentionScores, dim=-1) #  N × h × T × T

    A = AttentionWeights @ V #  N × h × T × d_v
    A = A.transpose(1,2).contiguous().view(N, T, self.n_heads*self.d_v ) #  N × T × h*d_v

    return self.fc(A)


In [4]:
class TransformerBlock(nn.Module):
  def __init__(self, d_k, d_v, d_model, n_heads, max_len, dropout_prob=0.2):
    super().__init__()

    self.ln1 = nn.LayerNorm(d_model)
    self.ln2 = nn.LayerNorm(d_model)
    self.mha = CausalSelfAttention(d_k, d_v, d_model, n_heads, max_len)
    self.ann = nn.Sequential(
        nn.Linear(d_model, d_model*3),
        nn.GELU(),
        nn.Linear(d_model*3, d_model),
        nn.Dropout(dropout_prob)
    )
    self.dropout = nn.Dropout(dropout_prob)

  def forward(self, x, pad_mask= None):
    x = self.ln1(x + self.mha(x,pad_mask))
    x = self.ln2(x + self.ann(x))
    x = self.dropout(x)
    return x

In [5]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, max_len = 2048, dropout_prob=0.2):
    super().__init__()
    self.dropout = nn.Dropout(dropout_prob)

    position = torch.arange(max_len).unsqueeze(1)
    exp_term = torch.arange(0, d_model, 2)
    div_term = torch.exp(exp_term * (-np.log(10000.0) / d_model))
    pe = torch.zeros(1, max_len, d_model)
    pe[0, :, 0::2] = torch.sin(position * div_term)
    pe[0, :, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe',pe)

  def forward(self, x):
    x = x+self.pe[:, :x.size(1), :]
    return self.dropout(x)

In [6]:
class Decoder(nn.Module):
  def __init__(self, vocab_size, max_len,d_model,d_k,d_v,
               n_heads,n_layers,dropout_prob):
    super().__init__()

    self.embedding = nn.Embedding(vocab_size,d_model)

    self.pos_encoding = PositionalEncoding(d_model, max_len, dropout_prob)

    transformer_blocks = [TransformerBlock(d_k, d_v,
                                           d_model,
                                           n_heads,
                                           max_len,
                                           dropout_prob) for i in range(n_layers)]

    self.transformer_blocks = nn.Sequential(*transformer_blocks)
    self.ln = nn.LayerNorm(d_model)
    self.fc = nn.Linear(d_model, vocab_size)

  def forward(self, x, pad_mask=None):
    x = self.embedding(x)
    x = self.pos_encoding(x)
    for block in self.transformer_blocks:
      x = block(x, pad_mask)


    x = self.ln(x)
    #many to many problem
    x = self.fc(x)

    return x

In [7]:
model = Decoder(vocab_size = 10000,
                max_len = 512,
                d_model = 64,
                d_k = 16,
                d_v = 16,
                n_heads = 4,
                n_layers = 2,
                dropout_prob = .1)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

Decoder(
  (embedding): Embedding(10000, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): CausalSelfAttention(
        (W_q): Linear(in_features=64, out_features=64, bias=True)
        (W_k): Linear(in_features=64, out_features=64, bias=True)
        (W_v): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=192, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=192, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, ele

##dummy test decoder

In [8]:
x = np.random.randint(0,10000, size=(8,512))
x = torch.tensor(x).to(device)

In [9]:
y = model(x)
x.shape, y.shape

(torch.Size([8, 512]), torch.Size([8, 512, 10000]))

In [10]:
mask = torch.ones((8,512))
mask[:,256:]= 0
mask = mask.to(device)

In [11]:
y = model(x, mask)
x.shape, y.shape

(torch.Size([8, 512]), torch.Size([8, 512, 10000]))

## Tokenization


In [12]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [13]:
raw_datasets = load_dataset('glue','sst2')
raw_datasets

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [14]:
def tokenize_fn(batch):
  return tokenizer(batch['sentence'], truncation = True)

In [15]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched =True)
tokenized_datasets

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [16]:
type(tokenized_datasets)

datasets.dataset_dict.DatasetDict

In [17]:
tokenized_datasets['train'][0]

{'sentence': 'hide new secretions from the parental units ',
 'label': 0,
 'idx': 0,
 'input_ids': [101, 4750, 1207, 3318, 5266, 1121, 1103, 22467, 2338, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [18]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=DistilBertTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}, padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [19]:
tokenized_datasets = tokenized_datasets.remove_columns(['sentence','idx','label'])
# tokenized_datasets = tokenized_datasets.rename_column('label','labels')
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [20]:
BATCH_SIZE = 32

train_loader = DataLoader(tokenized_datasets['train'],
                          batch_size=BATCH_SIZE,
                          shuffle=True,
                          collate_fn=data_collator)

valid_loader = DataLoader(tokenized_datasets['validation'],
                          batch_size=BATCH_SIZE,
                          collate_fn=data_collator)

In [21]:
for batch in train_loader:
  for k, v in batch.items():
    print('k:',k,'v.shape:',v.shape)
  break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


k: input_ids v.shape: torch.Size([32, 61])
k: attention_mask v.shape: torch.Size([32, 61])


In [22]:
tokenizer.vocab_size

28996

In [23]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

In [31]:
model = Decoder(vocab_size = tokenizer.vocab_size,
                max_len = 512,
                d_model = 64,
                d_k = 8,
                d_v = 8,
                n_heads = 6,
                n_layers = 4,
                dropout_prob = .1)

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model.to(device)

Decoder(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): CausalSelfAttention(
        (W_q): Linear(in_features=64, out_features=48, bias=True)
        (W_k): Linear(in_features=64, out_features=48, bias=True)
        (W_v): Linear(in_features=64, out_features=48, bias=True)
        (fc): Linear(in_features=48, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=192, bias=True)
        (1): GELU(approximate='none')
        (2): Linear(in_features=192, out_features=64, bias=True)
        (3): Dropout(p=0.1, inplace=False)
      )
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (ln1): LayerNorm((64,), eps=1e-05, ele

In [32]:
train_loader

In [33]:
for X in train_loader:
  print(X)
  break

{'input_ids': tensor([[  101,  1103,  1178,  ...,  1468,   119,   102],
        [  101,  6245,   102,  ...,     0,     0,     0],
        [  101,  2503,  3853,  ...,     0,     0,     0],
        ...,
        [  101, 19924, 19601,  ...,     0,     0,     0],
        [  101,  1103, 11769,  ...,     0,     0,     0],
        [  101,   170,  1514,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [34]:
tokenizer.pad_token_id

0

In [35]:
def TrainModel(model, train_loader, valid_loader, epochs):

  trainLoss = torch.zeros(epochs)
  testLoss  = torch.zeros(epochs)

  lossfun = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
  optimizer = torch.optim.Adam(model.parameters())


  for epoch in range(epochs):
    model.train()
    t0 = datetime.now()
    trainBatchLoss = []

    for batch in train_loader:
      batch = {k:v.to(device) for k,v in batch.items()}

      targets = batch['input_ids'].clone().detach()
      # print(targets)
      # print('________________')
      targets = torch.roll(targets, shifts=-1, dims=1)
      # print(targets)
      # print('________________')
      targets[:,-1] = tokenizer.pad_token_id
      # print(targets)
      # print('________________')

      yHat = model(batch['input_ids'], batch['attention_mask'])
      # print(yHat, yHat.shape)
      #check here for lossfun shapes https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
      loss = lossfun(yHat.transpose(2,1), targets) # N×T×V --> N×V×T


      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      trainBatchLoss.append(loss.item())

    trainLoss[epoch] = np.mean(trainBatchLoss)

    model.eval()

    testBatchLoss = []

    for batch in valid_loader:
      batch = {k:v.to(device) for k,v in batch.items()}

      with torch.no_grad(): # deactivates autograd
        yHat = model(batch['input_ids'], batch['attention_mask'])

        targets = batch['input_ids'].clone().detach()
        targets = torch.roll(targets, shifts=-1, dims=1)
        targets[:,-1] = tokenizer.pad_token_id

        loss = lossfun(yHat.transpose(2,1), targets)

      testBatchLoss.append(loss.item())

    # get loss and error rate from the test batch

    testLoss[epoch] = np.mean(testBatchLoss)

    # print(testBatchLoss)
    # print(testBatchAcc)

    # print('________________________')
    # print(trainLoss)
    # print(trainAcc)
    # print(testLoss)
    # print(testAcc)
    t = datetime.now() - t0

    print(f'''Epoch: {epoch}/{epochs},,
              Train Loss: {trainLoss[epoch].item():.4f},
              Test Loss: {testLoss[epoch].item():.4f}, Duration:{t}''')


  return trainLoss,testLoss,model

In [36]:
trainLoss,testLoss,model = TrainModel(model,
                                      train_loader,
                                      valid_loader,
                                      epochs=10)

Epoch: 0/10,,
              Train Loss: 6.0714,
              Test Loss: 5.7849, Duration:0:01:05.413816
Epoch: 1/10,,
              Train Loss: 5.2124,
              Test Loss: 5.7016, Duration:0:01:04.967964
Epoch: 2/10,,
              Train Loss: 4.8735,
              Test Loss: 5.7377, Duration:0:01:04.848534
Epoch: 3/10,,
              Train Loss: 4.6749,
              Test Loss: 5.7888, Duration:0:01:04.893672
Epoch: 4/10,,
              Train Loss: 4.5361,
              Test Loss: 5.8326, Duration:0:01:04.910717
Epoch: 5/10,,
              Train Loss: 4.4272,
              Test Loss: 5.8899, Duration:0:01:04.378172
Epoch: 6/10,,
              Train Loss: 4.3387,
              Test Loss: 5.9054, Duration:0:01:04.531136
Epoch: 7/10,,
              Train Loss: 4.2656,
              Test Loss: 5.9390, Duration:0:01:05.223878
Epoch: 8/10,,
              Train Loss: 4.2041,
              Test Loss: 5.9536, Duration:0:01:07.252069
Epoch: 9/10,,
              Train Loss: 4.1478,
       

In [37]:
test_loader = DataLoader(tokenized_datasets['test'],
                          batch_size=BATCH_SIZE,
                          collate_fn=data_collator)

In [39]:
for batch in test_loader:
  batch = {k:v.to(device) for k,v in batch.items()}

  with torch.no_grad():
    yHat = model(batch['input_ids'], batch['attention_mask'])
    print(yHat.shape)
    break

torch.Size([32, 47, 28996])


In [65]:
batch['input_ids'][0]

tensor([  101, 16782,  1940,  2737, 21575,  1104,  6739,  1105, 11688,   119,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0], device='cuda:0')

In [62]:
prediction_ids = torch.argmax(yHat[0,:,:], axis = -1)
prediction_ids

tensor([  170,  2742,  7147, 21575,   102,  1103,  1105,  1103,   102,   102,
         1122,  1103,  1103,  1103,   102,   102,   102,   102,   102,   102,
          102,   102,   102,   102,  1122,  1122,  1122,  1122,  1122,   102,
          102,   102,   102,   102,   102,   102,   102,   102,   102,   102,
          102,   102,   102,   102,   102,   102,   102], device='cuda:0')

In [63]:
#predicted sentence
tokenizer.decode(prediction_ids)

'a marriagechamash [SEP] the and the [SEP] [SEP] it the the the [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] it it it it it [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP] [SEP]'

In [64]:
# acutall sentence
tokenizer.decode(batch['input_ids'][0])

'[CLS] uneasy mishmash of styles and genres. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [56]:
prediction_ids[4]

tensor(1103, device='cuda:0')

In [66]:
batch['input_ids'][0,:5]

tensor([  101, 16782,  1940,  2737, 21575], device='cuda:0')

In [73]:
prompt = 'I love my'
tokenized_prompt = tokenizer(prompt, return_tensors = 'pt')
tokenized_prompt

{'input_ids': tensor([[ 101,  146, 1567, 1139,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}

In [74]:
output = model(tokenized_prompt['input_ids'][:,:-1].to(device),
               tokenized_prompt['attention_mask'][:,:-1].to(device))
output.shape

torch.Size([1, 4, 28996])

In [75]:
prediction_ids = torch.argmax(output[:,-1,:], axis = -1)
prediction_ids

tensor([2089], device='cuda:0')

In [76]:
tokenizer.decode(prediction_ids)

'lips'